In [1]:
import math

class FWD:
    def __init__(self, value, derivative=0):
        self.value = value
        self.derivative = derivative

    # z = x(θ) + y(θ), dz/dθ = dx/dθ + dy/dθ
    def __add__(self, other):
        if isinstance(other, FWD):
            return FWD(self.value + other.value, self.derivative + other.derivative)
        else:
            return FWD(self.value + other, self.derivative)
        
    def __radd__(self, other):
        return self.__add__(other)

    # z = x(θ) - y(θ), dz/dθ = dx/dθ - dy/dθ
    def __sub__(self, other):
        if isinstance(other, FWD):
            return FWD(self.value - other.value, self.derivative - other.derivative)
        else:
            return FWD(self.value - other, self.derivative)
        
    def __rsub__(self, other):
        return FWD(other - self.value, -self.derivative)

    # z = x(θ) * y(θ), dz/dθ = x(θ) * dy/dθ + y(θ) * dx/dθ
    def __mul__(self, other):
        if isinstance(other, FWD):
            return FWD(self.value * other.value, self.derivative * other.value + self.value * other.derivative)
        else:
            return FWD(self.value * other, self.derivative * other)

    def __rmul__(self, other):
        return self.__mul__(other)

    # z = x(θ) / y(θ), dz/dθ = (y(θ) * dx/dθ - x(θ) * dy/dθ) / (y(θ))^2
    def __truediv__(self, other):
        if isinstance(other, FWD):
            return FWD(self.value / other.value, (other.value * self.derivative - self.value * other.derivative) / (other.value ** 2))
        else:
            return FWD(self.value / other, self.derivative / other)

    def __rtruediv__(self, other):
        return FWD(other / self.value, -other * self.derivative / (self.value ** 2))

    # y = sin(x(θ)), dy/dθ = cos(x(θ)) * dx/dθ
    @staticmethod
    def sin(x):
        return FWD(math.sin(x.value), math.cos(x.value) * x.derivative)

    # y = cos(x(θ)), dy/dθ = -sin(x(θ)) * dx/dθ
    @staticmethod
    def cos(x):
        return FWD(math.cos(x.value), -math.sin(x.value) * x.derivative)

    # y = sqrt(x(θ)), dy/dθ = 0.5 / sqrt(x(θ)) * dx/dθ
    @staticmethod
    def sqrt(x):
        return FWD(math.sqrt(x.value), 0.5 / math.sqrt(x.value) * x.derivative)

In [ ]:
def forward(func):
    def gradient_func(x):
        dual_x = FWD(x, 1.0)
        result = func(dual_x)
        return result.value, result.derivative
    return gradient_func

def simple_function(x):
    result = FWD.sqrt(x) + 3*x + 5
    return result

if __name__ == "__main__":
    grad_func = forward(simple_function)
    f_val, f_prime = grad_func(4)
    print(f"f(4) = {f_val}")
    print(f"f'(4) = {f_prime}")

f(4) = 19.0
f'(4) = 3.25
